<a href="https://colab.research.google.com/github/HS587885/Studying-MachineLearning/blob/main/Tensorflow_imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 초기코드 - python
from keras import utils
from keras.layers import Dense
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,ZeroPadding2D,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
#from PIL import Image
import os, glob, numpy as np
from keras.models import load_model
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Activation, Dense
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB2

In [ ]:
# !unzip /content/drive/MyDrive/data/view_data/seg_pred.zip
# !unzip /content/drive/MyDrive/data/view_data/seg_train.zip
# !unzip /content/drive/MyDrive/data/view_data/seg_test.zip

!unzip /content/drive/MyDrive/data/cats_dogs.zip

In [ ]:
train_data_dir = '/content/view_data'
#test_data_dir = '/content/seg_test'

img_height=260
img_width=260
batch_size=32

train_datagen = ImageDataGenerator(
    #rescale=1./255,
    rotation_range=20,
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=0.4,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.9,1.1),
    validation_split=0.2) # set validation split

val_datagen = ImageDataGenerator(
    #rescale=1./255,
     validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=True,
    subset='training') # set as training data

validation_generator = val_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=False,
    subset='validation') # set as validation data


effi = EfficientNetB2(include_top=False, input_shape=(260,260,3))
model=Sequential()
model.add(effi)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))
model.summary()

# model = EfficientNetB4(include_top=True, input_shape=(380,380,3))
# model.summary()
# 새 섹션
#optimizer = SGD(lr=1e-4, momentum=0.9)
#optimizer RMSprop(lr=2e-5)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = './model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/efficientnetB2_dropout12.model'
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)
nb_epochs=50
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint,early_stopping]
    )

hist_df = pd.DataFrame(history.history)
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
!cp history.csv /content/history.csv
!cp -r model /content/model

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb2 (Functional  (None, 9, 9, 1408)        7768569   
 )                                                               
                                                                 
 global_average_pooling2d_3  (None, 1408)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_3 (Dropout)         (None, 1408)              0         
                                                                 
 dense_3 (Dense)             (None, 2)                 2818      
                                                                 
Total params: 7771387 (29.65 MB)
Trainable params: 7703812 (29.39 MB)
Non-trainable params: 67575 (263.